Here's the continuation. We use the TF Lite model to classify unseen pics to test how our model performs

In [77]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import os
import shutil

Right, so below we need to set parameters same as in the training file, see ober-pipe-clasifier-initial-training.ipynb

This below cell just uses the same params as what was used in training

In [60]:
img_width, img_height = 300, 300

class_names = ['damage_fracture_collapse', 'deposit_rubble_rubbish', 'joint_displacement', 'ok_condition', 'roots', 'water_present']

Now, let's bring out our TF lite model

In [54]:
TF_MODEL_FILE_PATH = 'initial-model.tflite'

interpreter = tf.lite.Interpreter(model_path=TF_MODEL_FILE_PATH)

In [55]:
interpreter.get_signature_list()

{'serving_default': {'inputs': ['sequential_1_input'], 'outputs': ['outputs']}}

In [56]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

A quick function, which opens the image for the model to look at

In [57]:
def open_img(img_path, img_height, img_width):
    img = tf.keras.utils.load_img(
        img_path, target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    return img_array 

# What happens here

We basically have a look at a picture. We return a list with the 'class' first and then the score.

In [68]:

def predict_and_classify_image(img):
    img_array = open_img(img, img_height, img_width)
    predictions_lite = classify_lite(sequential_1_input=img_array)['outputs']
    score_lite = tf.nn.softmax(predictions_lite)

    # print(
    #     "{} image most likely belongs to {} with a {:.2f} percent confidence."
    #     .format(img, class_names[np.argmax(score_lite)], 100 * np.max(score_lite))
    # )
    return class_names[np.argmax(score_lite)], 100 * np.max(score_lite)

We look at all these 5000 images now from the 'unsorted' input folder. It contains randomly exported frames from hours of CCTV footage.

We run the lite model on these and spit it each pic into the correct corresponding class (or at least this is our hope). if the pic shows fractured pipes it will be copied to
"./post-initial-training-classifier-output/damage_fracture_collapse"

In [79]:
input_directory = './unsorted-images-for-post-initial-classification'
output_directory = './post-initial-training-classifier-output'
for image in os.listdir(input_directory):
    image_to_classify = os.path.join(input_directory, image)
    prediction = predict_and_classify_image(image_to_classify)
    target_path = os.path.join(output_directory, prediction[0],image)
    shutil.copy(image_to_classify, target_path)

How did the model do? OK for what was expected. We will need to manually look at the outputs and train a new network, which combines the original training batch and the tidied up and (semi-) manually labelled outpus to train a bigger model of improved accuracy. We might add some new classes, too, as obviously the model gets really confused when it looks at our starting manhole, for example.